In [0]:
# Importing required libraries
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor as knn_reg
from sklearn.preprocessing import MinMaxScaler

In [2]:
#Mounting my Google Drive to access the datasets
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd drive/My\ Drive/Datasets/Astro_analytics

/content/drive/My Drive/Datasets/Astro_analytics


In [0]:
#Loading the training and test data as a DataFrame
train_data = pd.read_csv('train_techsoc.csv')
test_data = pd.read_csv('test_techsoc.csv')

In [5]:
train_data.head()

,id,epoch,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
0,0,2014-01-01T00:00:00.000,0,-20551.542730,30442.183966,-48103.896802,-2.107881,-8.838165,-4.692610,-20522.087697,30489.622269,-48134.673845,-2.106080,-8.830029,-4.697368
1,1,2014-01-01T00:46:43.000,0,-24524.197175,3758.914022,-56744.859999,-0.702215,-9.915381,-1.422103,-24495.949108,3827.474600,-56794.235471,-0.704799,-9.908881,-1.430627
2,2,2014-01-01T01:33:26.001,0,-24549.751725,-23625.618041,-56249.319584,0.643839,-9.393008,1.678213,-24533.911750,-23545.490605,-56325.644061,0.637910,-9.391303,1.668032
3,3,2014-01-01T02:20:09.001,0,-21230.174839,-47925.399825,-48085.396439,1.660679,-7.829425,3.996479,-21233.351704,-47848.266747,-48188.986571,1.653338,-7.832768,3.987640
4,4,2014-01-01T03:06:52.002,0,-15592.870340,-67135.125475,-34668.440812,2.303293,-5.847494,5.441308,-15616.695660,-67072.953982,-34792.578727,2.296041,-5.854192,5.435586


In [6]:
test_data.head()

,id,sat_id,epoch,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
0,2839,1,2014-01-16T00:00:09.858,-67817.397487,16157.508991,-17699.599623,-1.890826,-5.186361,2.522935
1,2840,1,2014-01-16T00:21:21.703,-69478.899262,9409.371850,-14309.037717,-0.713499,-5.405417,2.797437
2,2841,1,2014-01-16T00:42:33.548,-69610.589088,2454.957800,-10603.729002,0.516784,-5.509840,3.017606
3,2842,1,2014-01-16T01:03:45.392,-68136.264985,-4554.276294,-6654.676586,1.814434,-5.489129,3.179740
4,2843,1,2014-01-16T01:24:57.237,-64959.510617,-11446.670933,-2541.010712,3.196845,-5.321401,3.274442


In [7]:
train_data.describe()

,id,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
count,3.146030e+05,314603.000000,314603.000000,314603.000000,314603.000000,314603.000000,314603.000000,314603.000000,314603.000000,314603.000000,314603.000000,314603.000000,314603.000000,314603.000000
mean,6.162981e+05,308.729278,-2052.820412,906.284465,-200.225719,-0.008445,0.001073,-0.000765,-2197.599209,760.381594,-231.413668,-0.007167,0.001536,-0.000648
std,3.566068e+05,171.484273,58837.278378,59460.566408,47897.610082,6.137968,6.018011,4.906864,58868.459114,59429.125193,47965.281835,6.136245,6.020746,4.904675
min,0.000000e+00,0.000000,-584669.524947,-653228.105332,-412983.615760,-19.230725,-19.103044,-18.543417,-591861.747637,-656407.859386,-415958.552183,-19.129742,-19.270387,-17.501765
25%,3.067495e+05,166.000000,-30557.652232,-27053.603239,-18622.190752,-4.418387,-4.347204,-2.912028,-30550.842640,-27096.612330,-18522.272590,-4.422935,-4.345943,-2.893898
50%,6.157480e+05,312.000000,-1327.493266,1258.411675,56.194724,0.010026,-0.079010,0.022547,-1472.931073,1273.803460,65.623381,0.042415,-0.028811,0.028361
75%,9.258155e+05,461.000000,25978.947371,29281.534990,19092.559761,4.539527,4.312437,2.878250,25905.867274,29201.885432,19049.549399,4.530306,4.320935,2.866773
max,1.233166e+06,599.000000,464907.435517,493013.360838,457639.536795,18.429913,21.399374,20.060218,464580.401303,486460.128942,460715.741054,17.246956,18.503485,20.054406


In [0]:
to_drop = ['id']
tr_df = train_data.drop(to_drop, axis=1)

In [0]:
#Dropping the 'epoch' column as including them increases the model's error
def process(df):
    df['epoch'] = pd.to_datetime(df['epoch'])
    df = df.drop(['epoch'], axis=1)
    return df
tr_df = process(tr_df)
test_data = process(test_data)

In [10]:
tr_df.head()

,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
0,0,-20551.542730,30442.183966,-48103.896802,-2.107881,-8.838165,-4.692610,-20522.087697,30489.622269,-48134.673845,-2.106080,-8.830029,-4.697368
1,0,-24524.197175,3758.914022,-56744.859999,-0.702215,-9.915381,-1.422103,-24495.949108,3827.474600,-56794.235471,-0.704799,-9.908881,-1.430627
2,0,-24549.751725,-23625.618041,-56249.319584,0.643839,-9.393008,1.678213,-24533.911750,-23545.490605,-56325.644061,0.637910,-9.391303,1.668032
3,0,-21230.174839,-47925.399825,-48085.396439,1.660679,-7.829425,3.996479,-21233.351704,-47848.266747,-48188.986571,1.653338,-7.832768,3.987640
4,0,-15592.870340,-67135.125475,-34668.440812,2.303293,-5.847494,5.441308,-15616.695660,-67072.953982,-34792.578727,2.296041,-5.854192,5.435586


In [0]:
test_df = test_data.copy()
new_df = tr_df.copy()

In [0]:
#Defining a Scaler function to scale the various columns of the training and test test, it improves the model's performance
#Because of scaling, each columns contributes equally to the prediction as every column's value lies between 0 and 1
scaler = MinMaxScaler()

In [0]:
#Scaling every column in both training and test set
def process_2(s):
    new_df[s] = pd.DataFrame(scaler.fit_transform(new_df[s].values.reshape(-1,1)))
    test_df[s] = pd.DataFrame(scaler.transform(test_df[s].values.reshape(-1,1)))

l = ['x_sim','y_sim','z_sim','Vx_sim','Vy_sim','Vz_sim']
for i in l:
    process_2(i)

In [14]:
test_df.head()

,id,sat_id,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
0,2839,1,0.496046,0.588489,0.454284,0.473900,0.372851,0.533193
1,2840,1,0.494474,0.582585,0.458151,0.506265,0.367052,0.540502
2,2841,1,0.494349,0.576499,0.462378,0.540085,0.364287,0.546365
3,2842,1,0.495745,0.570366,0.466882,0.575758,0.364836,0.550682
4,2843,1,0.498752,0.564336,0.471575,0.613761,0.369276,0.553204


In [15]:
new_df.head()

,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
0,0,-20551.542730,30442.183966,-48103.896802,-2.107881,-8.838165,-4.692610,0.540815,0.601030,0.419567,0.467983,0.276391,0.340940
1,0,-24524.197175,3758.914022,-56744.859999,-0.702215,-9.915381,-1.422103,0.537053,0.577700,0.409690,0.506504,0.247830,0.427923
2,0,-24549.751725,-23625.618041,-56249.319584,0.643839,-9.393008,1.678213,0.537017,0.553749,0.410224,0.543415,0.261532,0.510430
3,0,-21230.174839,-47925.399825,-48085.396439,1.660679,-7.829425,3.996479,0.540142,0.532485,0.419505,0.571329,0.302792,0.572194
4,0,-15592.870340,-67135.125475,-34668.440812,2.303293,-5.847494,5.441308,0.545458,0.515663,0.434786,0.588997,0.355171,0.610748


 

*   Here, I am using KNN (k-nearest neighbours) regression to predict the co-ordinates and velocities of the satellite.

*   First, I am selecting the entries which have same satellite ids - (i.e) essentially selecting a particular satellite and then predicting the co-ordinates and velocities of that particular satellite at different time using KNN algorithm.

*   Here, I set K's value to be 2, as MSE for k=2 is lesser when compared to any other values of K.





In [0]:
#Predicting X
pred_x = pd.DataFrame()
tmp = pd.DataFrame()

for sat in test_df['sat_id'].unique():
    train_X = new_df[new_df['sat_id']==sat][['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']]
    test_X = test_df[test_df['sat_id']==sat][['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']]
    target = new_df[new_df['sat_id']==sat][['x']]
    model = knn_reg(n_neighbors=2,weights='distance')
    model.fit(train_X, target)
    test_pred = model.predict(test_X)
    pred = pd.DataFrame()
    pred['sat_id'] = [sat for i in range(test_X.shape[0])]
    pred['x'] = test_pred
    pred_x = pred_x.append(pred)
    pred_x = pred_x.reset_index(drop=True)

    tmp2 = pd.DataFrame()
    tmp2['id'] = test_df[test_df['sat_id']==sat]['id']
    tmp = tmp.append(tmp2)
    tmp = tmp.reset_index(drop=True)

In [0]:
#Predicting Y
pred_y = pd.DataFrame()

for sat in test_df['sat_id'].unique():
    train_X = new_df[new_df['sat_id']==sat][['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']]
    test_X = test_df[test_df['sat_id']==sat][['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']]
    target = new_df[new_df['sat_id']==sat][['y']]
    model = knn_reg(n_neighbors=2,weights='distance')
    model.fit(train_X, target)
    test_pred = model.predict(test_X)
    pred = pd.DataFrame()
    pred['sat_id'] = [sat for i in range(test_X.shape[0])]
    pred['y'] = test_pred
    pred_y = pred_y.append(pred)
    pred_y = pred_y.reset_index(drop=True)


In [0]:
#Predicting Z
pred_z = pd.DataFrame()

for sat in test_df['sat_id'].unique():
    train_X = new_df[new_df['sat_id']==sat][['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']]
    test_X = test_df[test_df['sat_id']==sat][['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']]
    target = new_df[new_df['sat_id']==sat][['z']]
    model = knn_reg(n_neighbors=2,weights='distance')
    model.fit(train_X, target)
    test_pred = model.predict(test_X)
    pred = pd.DataFrame()
    pred['sat_id'] = [sat for i in range(test_X.shape[0])]
    pred['z'] = test_pred
    pred_z = pred_z.append(pred)
    pred_z = pred_z.reset_index(drop=True)

In [0]:
#Predicting Vx
pred_vx = pd.DataFrame()

for sat in test_df['sat_id'].unique():
    train_X = new_df[new_df['sat_id']==sat][['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']]
    test_X = test_df[test_df['sat_id']==sat][['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']]
    target = new_df[new_df['sat_id']==sat][['Vx']]
    model = knn_reg(n_neighbors=2,weights='distance')
    model.fit(train_X, target)
    test_pred = model.predict(test_X)
    pred = pd.DataFrame()
    pred['sat_id'] = [sat for i in range(test_X.shape[0])]
    pred['vx'] = test_pred
    pred_vx = pred_vx.append(pred)
    pred_vx = pred_vx.reset_index(drop=True)

In [0]:
#Predicting Vy
pred_vy = pd.DataFrame()

for sat in test_df['sat_id'].unique():
    train_X = new_df[new_df['sat_id']==sat][['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']]
    test_X = test_df[test_df['sat_id']==sat][['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']]
    target = new_df[new_df['sat_id']==sat][['Vy']]
    model = knn_reg(n_neighbors=2,weights='distance')
    model.fit(train_X, target)
    test_pred = model.predict(test_X)
    pred = pd.DataFrame()
    pred['sat_id'] = [sat for i in range(test_X.shape[0])]
    pred['vy'] = test_pred
    pred_vy = pred_vy.append(pred)
    pred_vy = pred_vy.reset_index(drop=True)

In [0]:
#Predicting Vz
pred_vz = pd.DataFrame()

for sat in test_df['sat_id'].unique():
    train_X = new_df[new_df['sat_id']==sat][['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']]
    test_X = test_df[test_df['sat_id']==sat][['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']]
    target = new_df[new_df['sat_id']==sat][['Vz']]
    model = knn_reg(n_neighbors=2,weights='distance')
    model.fit(train_X, target)
    test_pred = model.predict(test_X)
    pred = pd.DataFrame()
    pred['sat_id'] = [sat for i in range(test_X.shape[0])]
    pred['vz'] = test_pred
    pred_vz = pred_vz.append(pred)
    pred_vz = pred_vz.reset_index(drop=True)

In [0]:
#Writing the predictions to a CSV file.
sample = pd.DataFrame()
sample['id'] = tmp['id']
sample['x'] = pred_x['x']
sample['y'] = pred_y['y']
sample['z'] = pred_z['z']
sample['Vx'] = pred_vx['vx']
sample['Vy'] = pred_vy['vy']
sample['Vz'] = pred_vz['vz']

sample.to_csv('submissions.csv', index=False)